In [10]:
import pandas as pd
import string

class Slovenian_preprocessor():

    def __init__(self, data):
        self.data = data
    
    def preprocess(self):
        self.data['label'] = self.data['label'].astype(str)
        text = ''
        overall_entities = []
        entities = []
        current_entity = ''
        prev_word=''
        for _, row in self.data.iterrows():
            # print(f"word: {row['word']}, label: {row['label']}, prev_word: {prev_word}")
            if prev_word == '.' and len(text.split())>5:
                if current_entity:
                    entities.append({'entity': current_entity})
                    current_entity = ''
                # print(f"ENTRO NELLA COND")
                overall_entities.append({'text': text, 'entities': entities})
                text = ''
                entities = []
            word, label = row['word'], row['label']
            if text!='' or (word.strip() in string.punctuation) or (prev_word.strip() in string.punctuation):
                space = ' '
            else:
                space = ''
            text = text + space + word
            if label != 'O':
                if label.startswith('B-'):
                    if current_entity:
                        entities.append({'entity': current_entity})
                    current_entity = word
                elif label.startswith('I-') and current_entity:
                    current_entity += ' ' + word
            prev_word = word
        data_df = pd.DataFrame(columns=['text', 'entities'])
        for el in overall_entities:
            el['text'] = el['text'].strip() 
            data_df = pd.concat([data_df, pd.DataFrame([el])], ignore_index=True)
        self.data = data_df

In [12]:
data_train = pd.read_csv('/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/data/slovenian/E3C_Slovenian_Train_SL_L1.csv', header=None, names=['word', 'label'])
preprocessor = Slovenian_preprocessor(data_train)
preprocessor.preprocess()
preprocessor.data

# data_test = pd.read_csv('/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/data/slovenian/E3C_Slovenian_Test_SL_L1.csv', header=None, names=['word', 'label'])
# preprocessor = Slovenian_preprocessor(data_test)
# preprocessor.preprocess()
# preprocessor.data

,text,entities
0,61 - letni moški se je pojavil s kroničnim lev...,"[{'entity': 'levim ušesom izcedekom'}, {'entit..."
1,MRI s kontrastom je pokazal asimetrične signal...,"[{'entity': 'signalne spremembe'}, {'entity': ..."
2,- 8 . živčnega kompleksa .,[]
3,pacient je bil zdravljen konzervativno z lokal...,"[{'entity': 'vrtoglavice'}, {'entity': 'diplop..."
4,"22 - letna ženska , brez prej diagnosticiranih...","[{'entity': 'dihalne stiske'}, {'entity': 'utr..."
...,...,...
663,dvaindvajsetletna ženska iz šrilanke z anamnez...,[{'entity': 'akutni vnetni demielinizirajoči p...
664,študija prevodnosti živca je razkrila varianto...,"[{'entity': 'AIDP'}, {'entity': 'guillain-barr..."
665,analiza likvorja po 2 tednih je bila normalna ...,[]
666,ona je bila zdravljena z intravenskim imunoglo...,[]


In [17]:
max_length = 0
for i, row in preprocessor.data.iterrows():
    if len(row['text'] + ' ' + str(row['entities'])) > max_length:
        max_length = len(row['text'] + ' ' + str(row['entities']))
        max_row = row

In [18]:
max_length

1012